In [ ]:
import os
from pathlib import Path
import sys

import torch

rootDir = Path(os.path.abspath(__name__)).parent.parent
sys.path.append(str(rootDir))

from commsimcnn.util import get_conv2d_output_shape

In [ ]:
get_conv2d_output_shape(x=torch.Size([20, 30]), kernel_size=3)

In [ ]:
test = torch.rand([1, 28, 28])
test_shape = test.shape

In [ ]:
from torch import nn

test_conv_layer = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=3,
            stride=1,
            padding=0
        )

test_max_pool = nn.MaxPool2d(kernel_size=2)

In [ ]:
test_max_pool(test).shape

In [ ]:
get_conv2d_output_shape(test_shape, kernel_size=2)

In [ ]:
from commsimcnn.util import get_output_shape